In [2]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [3]:
def plot(img):
    plt.figure()
    plt.imshow(img[:, 40, :], cmap='gray')
    
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [4]:
run_dirs = glob.glob('../runs/*/predict')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 1 runs 

0 ../runs/vae_20190417_1843_gpu=6_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=1.0_lw=[1_1]_tag=brains_test/predict 



In [8]:
# set run dir
run_dir = run_dirs[0]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), '*', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    #print(i, os.path.basename(pre_dir))
    print(i, pre_dir.split('/')[-2] + '/' + pre_dir.split('/')[-1])

found 1 pre_dirs
0 test/500_04


In [9]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[0]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)

print('found meta.csv: {}'.format(csv.shape))

found meta.csv: (271, 37)


In [53]:
# classifier
clf_dir = '../../../../backup/agemorph/models/clf/clf_20190330_1549_gpu=4_bs=8_lr=0.001_b1=0.9_b2=0.999_ep=0.1_bn=True_ds=1_lw=[1.0]/'
model_name = 'clf_200'
model_file = os.path.join(clf_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')

found model file


In [11]:
# gpu config
gpu_id = 7

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [54]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/vae_20190417_1843_gpu=6_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_lr=0.0001_pl=100_is=0.01_vr=1.0_lw=[1_1]_tag=brains_test/predict/test/500_04/clf_200.csv


In [55]:
clf_csvs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_csvs = sorted(clf_csvs)

for i, clf_csv in enumerate(clf_csvs):
    print(i, os.path.basename(clf_csv))

0 clf_100.csv
1 clf_200.csv


In [56]:
csv_path = clf_csvs[1]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 41)


In [58]:
def acc_threshold(csv, col, t):
    true_pos = ((csv[col] > t) & (csv.pat_dx_1 == 3)).sum()
    true_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 2)).sum()
    false_pos = ((csv[col] > t) & (csv.pat_dx_1 == 2)).sum()
    false_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 3)).sum()

    acc_p = true_pos / (true_pos + false_neg)
    acc_s = true_neg / (true_neg + false_pos)

    balanced_acc = (acc_p + acc_s) / 2

    true = true_pos + true_neg
    false = false_pos + false_neg
    
    return (round(balanced_acc, 3), true, false, round(acc_p, 2), round(acc_s, 2), round(t, 2))

In [59]:
def f1_threshold(csv, col, t):
    true_pos = ((csv[col] > t) & (csv.pat_dx_1 == 3)).sum()
    true_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 2)).sum()
    false_pos = ((csv[col] > t) & (csv.pat_dx_1 == 2)).sum()
    false_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 3)).sum()

    prec = true_pos / (true_pos + false_pos)
    reca = true_pos / (true_pos + false_neg)
    
    f1 = 2 * (prec * reca) / (prec + reca)

    true = true_pos + true_neg
    false = false_pos + false_neg
    
    return (round(f1, 3), true, false, round(prec, 2), round(reca, 2), round(t, 2))

In [60]:
def best_threshold_acc(csv, col):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        l.append(acc_threshold(csv, col, t))
        
    return max(l)

In [61]:
def best_threshold_f1(csv, col):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        l.append(f1_threshold(csv, col, t))
        
    return max(l)

In [62]:
def acc_cv(csv_in, col, cv_split):
    
    csv = csv_in[csv_in.cv_split != cv_split]
    
    t = best_threshold_acc(csv, col)[5]
    print(t)
    
    csv = csv_in[csv_in.cv_split == cv_split]
    
    acc = acc_threshold(csv, col, t)
    print(acc)
    
    return acc

In [63]:
def f1_cv(csv_in, col, cv_split):
    
    csv = csv_in[csv_in.cv_split != cv_split]
    
    t = best_threshold_f1(csv, col)[5]
    print(t)
    
    csv = csv_in[csv_in.cv_split == cv_split]
    
    f1 = f1_threshold(csv, col, t)
    print(f1)
    
    return f1

In [80]:
# create 5 folds
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_splits = np.split(pMCI, np.linspace(0, len(pMCI), 6).astype(int)[1:-1])
s_splits = np.split(sMCI, np.linspace(0, len(sMCI), 6).astype(int)[1:-1])

In [81]:
# write folds to csv
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [82]:
# F1 for x
l = [f1_cv(csv, 'pAD_xr', i)[0] for i in range(5)]

print('\nf1_xr:', round(np.array(l).mean(), 3))

0.01
(0.609, 35, 18, 0.52, 0.74, 0.01)
0.01
(0.558, 36, 19, 0.52, 0.6, 0.01)
0.01
(0.703, 42, 11, 0.72, 0.68, 0.01)
0.01
(0.5, 33, 22, 0.46, 0.55, 0.01)
0.01
(0.55, 37, 18, 0.55, 0.55, 0.01)

f1_xr: 0.584


In [83]:
#F1 y_fake
l = [f1_cv(csv, 'pAD_yf', i)[0] for i in range(5)]

print('\nf1_yf:', round(np.array(l).mean(), 3))

0.02
(0.51, 28, 25, 0.41, 0.68, 0.02)
0.02
(0.556, 31, 24, 0.44, 0.75, 0.02)
0.02
(0.762, 43, 10, 0.7, 0.84, 0.02)
0.02
(0.643, 35, 20, 0.5, 0.9, 0.02)
0.02
(0.615, 35, 20, 0.5, 0.8, 0.02)

f1_yf: 0.617


In [84]:
# F1 y_real
l = [f1_cv(csv, 'pAD_yr', i)[0] for i in range(5)]

print('\nf1_yr:', round(np.array(l).mean(), 3))

0.35
(0.591, 35, 18, 0.52, 0.68, 0.35)
0.35
(0.653, 38, 17, 0.55, 0.8, 0.35)
0.01
(0.766, 42, 11, 0.64, 0.95, 0.01)
0.01
(0.679, 38, 17, 0.55, 0.9, 0.01)
0.02
(0.653, 38, 17, 0.55, 0.8, 0.02)

f1_yr: 0.668


In [85]:
# balanced acc x_real
l = [acc_cv(csv, 'pAD_xr', i)[0] for i in range(5)]

np.array(l).mean()

0.01
(0.677, 35, 18, 0.74, 0.62, 0.01)
0.01
(0.643, 36, 19, 0.6, 0.69, 0.01)
0.01
(0.769, 42, 11, 0.68, 0.85, 0.01)
0.01
(0.589, 33, 22, 0.55, 0.63, 0.01)
0.01
(0.646, 37, 18, 0.55, 0.74, 0.01)


0.6648

In [86]:
# balanced acc y_fake
l = [acc_cv(csv, 'pAD_yf', i)[0] for i in range(5)]

np.array(l).mean()

0.02
(0.563, 28, 25, 0.68, 0.44, 0.02)
0.02
(0.604, 31, 24, 0.75, 0.46, 0.02)
0.59
(0.652, 38, 15, 0.42, 0.88, 0.59)
0.02
(0.693, 35, 20, 0.9, 0.49, 0.02)
0.02
(0.671, 35, 20, 0.8, 0.54, 0.02)


0.6365999999999999

In [87]:
# balanced acc y_real
l = [acc_cv(csv, 'pAD_yr', i)[0] for i in range(5)]

np.array(l).mean()

0.35
(0.666, 35, 18, 0.68, 0.65, 0.35)
0.35
(0.714, 38, 17, 0.8, 0.63, 0.35)
0.01
(0.827, 42, 11, 0.95, 0.71, 0.01)
0.89
(0.739, 42, 13, 0.65, 0.83, 0.89)
0.89
(0.7, 40, 15, 0.6, 0.8, 0.89)


0.7292